In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
books = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/books.csv')

In [4]:
books.fillna('',inplace= True)

In [5]:
books['tags'] = books['title']+' '+books['author']+' '+ books['genres']+' '+books['description']

In [6]:
vector = TfidfVectorizer(stop_words='english')
vector_matrix = vector.fit_transform(books['tags'])

In [7]:
similarity = cosine_similarity(vector_matrix,vector_matrix)

In [22]:
def recommend(input,top_n=5):
  #converting input into a vector
  input_vect = vector.transform([input])
  #checking similarity
  similarity_score = cosine_similarity(input_vect,vector_matrix).flatten()

  #creating a score dataframe that contains similarity scores
  score_df = pd.DataFrame({
      'title': books['title'],
      'author':books['author'],
      'genres':books['genres'],
      'avg_rating':books['avg_rating'],
      'ratings_count':books['num_ratings'],
      'similarity_score':similarity_score
  })
  #changing string into number
  score_df['avg_rating'] = pd.to_numeric(score_df['avg_rating'], errors='coerce')
  score_df['ratings_count'] = pd.to_numeric(score_df['ratings_count'], errors='coerce')
  score_df.fillna(0, inplace=True)
  #normalizing
  score_df['ratings_count'] = score_df['ratings_count']/score_df['ratings_count'].max()
  #since avg_rating and ratings count matters as much as similarity
  score_df['final_score'] =((score_df['similarity_score']*0.5)+(score_df['avg_rating']/5*0.3)+(score_df['ratings_count']*0.2))

  top_books = score_df.sort_values('final_score',ascending=False).head(top_n)
  return top_books[['title','author','genres','avg_rating']]

In [27]:
recommend('')

,title,author,genres,avg_rating
6914,The Raj Quartet,Paul Scott,"['Fiction', 'Historical Fiction', 'India', 'Hi...",4.48
457,The Chain Between Worlds (The Lost Artefacts #1),Johnathon Nicolaou,"['Fantasy', 'Young Adult', 'Urban Fantasy', 'F...",4.62
2788,Living The Best Day Ever,Hendri Coetzee,"['Nonfiction', 'Adventure']",4.34
1466,"Hometown Girl After All (Hometown, #2)",Kirsten Fullmer,"['Contemporary', 'Young Adult', 'New Adult', '...",4.32
3022,"Hometown Girl After All (Hometown, #2)",Kirsten Fullmer,"['Contemporary', 'Young Adult', 'New Adult', '...",4.32
